In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np        
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.models import load_model, Model
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
sales = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
item_cate = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
sample_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
test_data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
sales['item_cnt_day'].value_counts().head(30)

In [ ]:
items.item_id.nunique()
item_cate.isnull().sum()

In [ ]:
items

In [ ]:
sales = sales.query('item_price > 0')
sales = sales[sales['shop_id'].isin(test_data['shop_id'].unique())]
sales = sales[sales['item_id'].isin(test_data['item_id'].unique())]
sales = sales.query('item_cnt_day >= 0 and item_price < 75000')
sales['year'] = pd.to_datetime(sales['date']).dt.strftime('%Y')
sales['month'] = pd.to_datetime(sales['date']).dt.strftime('%m')
sales.head(5)

In [ ]:
#satışlarımızı sales train içindeki columnları grup haline getiriyoruz
monthSales=sales.groupby(["date_block_num","shop_id","item_id"])\
                [["date_block_num","date","item_price","item_cnt_day"]]\
                    .agg({"date_block_num":'mean',"date":["min",'max'],\
                                  "item_price":"mean","item_cnt_day":"sum"})

monthSales.head()

In [ ]:
sales_flat = monthSales.item_cnt_day.apply(list).reset_index()
sales_flat.head(10)

In [ ]:
sales_flat = pd.merge(test_data,sales_flat,on = ['item_id','shop_id'],how = 'left')
sales_flat.head(10)
#test_datadaki veriler ile sales_flat verileri merge edilir


#boş değerleri 0 ile dolduruyoruz
sales_flat.fillna(0,inplace = True)
sales_flat.drop(['shop_id','item_id'],inplace = True, axis = 1)
sales_flat.head(10)

In [ ]:
sales2 = sales_flat.pivot_table(index='ID', columns='date_block_num',fill_value = 0,aggfunc='sum' )
sales2.head(10)

In [ ]:

X_train = np.expand_dims(sales2.values[:,:-1],axis = 2)

y_train = sales2.values[:,-1:]

X_test = np.expand_dims(sales2.values[:,1:],axis = 2)

 
print(X_train.shape,y_train.shape,X_test.shape)

In [ ]:
y_train

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0001)

In [ ]:
sales_model = Sequential()
sales_model.add(LSTM(units = 128,input_shape = (33,1), return_sequences=True))
sales_model.add(Dropout(0.3))
sales_model.add(LSTM(64))
sales_model.add(Dropout(0.2))
sales_model.add(Dense(1))

sales_model.compile(loss = 'mse',optimizer = optimizer,\
                            metrics = ['mean_squared_error'])
sales_model.summary()

In [ ]:
callback = 

In [ ]:
historySales=sales_model.fit(X_train,y_train,batch_size = 64,epochs = 30)

In [ ]:
sonuc = sales_model.predict(X_test) 
ss = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':sonuc.ravel()})


In [ ]:
ss.to_csv('./submission.csv', index=False)

In [ ]:
plt.plot(historySales.history['loss'], color='b', label="Training loss")
plt.legend(loc='best', shadow=True)
#Loss değerimizin düşüşü

In [ ]:
plt.plot(ss["ID"],ss["item_cnt_month"], color='b', label="Eşya satışları")
